In [17]:
# input: book name data csv
# output: book prices in different formats but without "rank"
# need to scrape url, too

In [20]:
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from tqdm.autonotebook import tqdm

In [3]:
book_data = pd.read_csv("sampled_book_data.csv")
book_titles = book_data["title"].tolist()

In [7]:
def set_up_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    return webdriver.Chrome(service=Service(r"C:\Users\USER\OneDrive\桌面\C2L2\chromedriver.exe"), options=chrome_options)

In [15]:
outcomes = pd.DataFrame(columns=["title", "publication_date", "KINDLE", "KINDLE_BASE", "AUDIO_DOWNLOAD", "HARDCOVER", "HARDCOVER_BASE", "PAPERBACK"])

In [12]:
a = set_up_driver(False)
a.get("https://www.amazon.com/s?k=harry potter")
a.page_source

'<html lang="zh-tw" class="a-ws a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-hires a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.2-2024-03-28"><!-- sp:feature:head-start --><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js" crossorigin="anonymous"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv="x-dns-prefetch-control" content="on">\n<link rel="dns-prefetch" href="https://images-na.ssl-images

In [23]:
def extract_prices(driver: webdriver.Chrome, book_title: str) -> pd.DataFrame:
    try:
        input = driver.find_element(By.ID, "e")
        input.send_keys(book_title)
        button = driver.find_element(By.ID, "f")
        button.click()
    except:
        pass

    title = driver.find_element(By.CLASS_NAME, "a-size-mini.a-spacing-none.a-color-base.s-line-clamp-2").find_element(By.TAG_NAME, "a")
    title.click()
    sleep(1)
    driver.refresh()

    # publication date
    print("----------------------------------")
    publication_date = None
    try:
        publication_date = (
            driver.find_element(By.ID, "rpi-attribute-book_details-publication_date").find_elements(By.TAG_NAME, "div")[2].find_element(By.TAG_NAME, "span").get_attribute("innerHTML").strip()
        )
        print("publication_date operation 1 successful")
    except:
        print("Error in publication_date operation 1")
    try:
        publication_date = (
            driver.find_element(By.ID, "rpi-attribute-audiobook_details-release-date").find_elements(By.TAG_NAME, "div")[2].find_element(By.TAG_NAME, "span").get_attribute("innerHTML").strip()
        )
        print("publication_date operation 2 successful")
    except:
        print("Error in publication_date operation 2")
    print("publication_date: ", publication_date)

    # Kindle
    print("----------------------------------")
    driver.find_element(By.ID, "formats").find_element(By.ID, "tmm-grid-swatch-KINDLE").click()
    sleep(1)
    form = driver.find_element(By.ID, "priceBlock-outsideOfForm_feature_div")

    # KINDLE_BASE
    KINDLE_BASE = None
    try:
        KINDLE_BASE = form.find_element(By.ID, "basis-price").get_attribute("innerHTML").lstrip().rstrip(" <!-- Omnibus - More Information Tooltip -->\n")
        print("KINDLE_BASE operation 1 successful")
    except:
        print(f"Error in KINDLE_BASE operation 1")
    print("KINDLE_base: ", KINDLE_BASE)

    # KINDLE
    KINDLE = None
    try:
        KINDLE = form.find_element(By.ID, "kindle-price").get_attribute("innerHTML").strip()
        print("KINDLE operation 1 successful")
    except:
        print(f"Error in KINDLE operation 1")
    print("KINDLE: ", KINDLE)

    # Hardcover
    print("----------------------------------")
    driver.find_element(By.ID, "formats").find_element(By.ID, "tmm-grid-swatch-HARDCOVER").click()
    sleep(1)

    # HARDCOVER
    HARDCOVER = None
    try:
        HARDCOVER = driver.find_element(By.ID, "booksHeaderSection").find_element(By.ID, "price").get_attribute("innerHTML").strip()
        print("HARDCOVER operation 1 successful")
    except:
        print(f"Error in HARDCOVER operation 1")
    try:
        HARDCOVER = (
            driver.find_element(By.CLASS_NAME, "a-box-inner.a-accordion-row-container").find_element(By.CLASS_NAME, "a-offscreen").find_element(By.ID, "price").get_attribute("innerHTML").strip()
        )
        print("HARDCOVER operation 2 successful")
    except:
        print(f"Error in HARDCOVER operation 2")
    print("HARDCOVER: ", HARDCOVER)

    # HARDCOVER_BASE
    HARDCOVER_BASE = None
    try:
        HARDCOVER_BASE = driver.find_element(By.CLASS_NAME, "a-accordion-inner.accordion-row-content").find_element(By.ID, "listPrice").get_attribute("innerHTML").strip()
        print("HARDCOVER_BASE operation 1 successful")
    except:
        print(f"Error in HARDCOVER_BASE operation 1")
    try:
        HARDCOVER_BASE = driver.find_element(By.ID, "booksAdditionalPriceInfoContainer").find_element(By.ID, "listPrice").get_attribute("innerHTML").strip()
        print("HARDCOVER_BASE operation 2 successful")
    except:
        print(f"Error in HARDCOVER_BASE operation 2")
    print("HARDCOVER_BASE: ", HARDCOVER_BASE)

    # AUDIODOWNLOAD
    print("----------------------------------")
    AUDIO_DOWNLOAD = None
    try:
        AUDIO_DOWNLOAD = (
            driver.find_element(By.ID, "formats")
            .find_element(By.ID, "tmm-grid-swatch-AUDIO_DOWNLOAD")
            .find_element(By.CLASS_NAME, "slot-price")
            .find_element(By.TAG_NAME, "span")
            .find_element(By.TAG_NAME, "span")
            .get_attribute("innerHTML")
            .strip()
        )
        print("AUDIO_DOWNLOAD operation 1 successful")
    except:
        print(f"Error in AUDIO_DOWNLOAD operation 1")
    print("AUDIO_DOWNLOAD: ", AUDIO_DOWNLOAD)

    # Paperback
    print("----------------------------------")
    PAPERBACK = None
    try:
        PAPERBACK = (
            driver.find_element(By.ID, "formats")
            .find_element(By.ID, "tmm-grid-swatch-PAPERBACK")
            .find_element(By.CLASS_NAME, "slot-price")
            .find_element(By.TAG_NAME, "span")
            .get_attribute("innerHTML")
            .strip()
        )
        print("PAPERBACK operation 1 successful")
    except:
        print(f"Error in PAPERBACK operation 1")
    print("PAPERBACK: ", PAPERBACK)

    # AUDIOBOOK
    print("----------------------------------")
    AUDIOBOOK = None
    try:
        AUDIOBOOK = (
            driver.find_element(By.ID, "formats")
            .find_element(By.ID, "tmm-grid-swatch-AUDIOBOOK")
            .find_element(By.CLASS_NAME, "slot-price")
            .find_element(By.TAG_NAME, "span")
            .find_element(By.TAG_NAME, "span")
            .get_attribute("innerHTML")
            .strip()
        )
        print("AUDIOBOOK operation 1 successful")
    except:
        print(f"Error in AUDIOBOOK operation 1")
    print("AUDIOBOOK: ", AUDIOBOOK)

    new_data_prices = pd.DataFrame(
        {
            "title": [book_title],
            "publication_date": [publication_date],
            "KINDLE": [KINDLE],
            "KINDLE_BASE": [KINDLE_BASE],
            "AUDIO_DOWNLOAD": [AUDIO_DOWNLOAD],
            "HARDCOVER_BASE": [HARDCOVER_BASE],
            "PAPERBACK": [PAPERBACK],
            "AUDIOBOOK": [AUDIOBOOK],
        }
    )

    return new_data_prices

In [26]:
for book_title in tqdm(book_titles[:5]):
    search_result_link = f"https://www.amazon.com/s?k={book_title.replace(' ', '+')}"
    print(f"==================================\nbook: {book_title}\nurl: {search_result_link}")
    driver = set_up_driver(False)
    driver.get(search_result_link)
    sleep(1)
    driver.refresh()
    sleep(1)
    new_outcome = extract_prices(driver, book_title)
    driver.quit()
    outcomes = pd.concat([outcomes, new_outcome])
outcomes.to_csv("output2.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

book: Fantastic Beasts and Where to Find Them
url: https://www.amazon.com/s?k=Fantastic Beasts and Where to Find Them
----------------------------------
publication_date operation 1 successful
Error in publication_date operation 2
publication_date:  2017年 11月 7日
----------------------------------


  0%|          | 0/5 [00:48<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="priceBlock-outsideOfForm_feature_div"]"}
  (Session info: chrome=123.0.6312.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF719924C82+3505170]
	(No symbol) [0x00007FF719550852]
	(No symbol) [0x00007FF719404145]
	(No symbol) [0x00007FF719449ADD]
	(No symbol) [0x00007FF719449C1C]
	(No symbol) [0x00007FF71948AB27]
	(No symbol) [0x00007FF71946BECF]
	(No symbol) [0x00007FF7194883B2]
	(No symbol) [0x00007FF71946BC33]
	(No symbol) [0x00007FF71943D618]
	(No symbol) [0x00007FF71943E6B1]
	GetHandleVerifier [0x00007FF7199567DD+3708781]
	GetHandleVerifier [0x00007FF7199AFC5D+4074477]
	GetHandleVerifier [0x00007FF7199A7DDF+4042095]
	GetHandleVerifier [0x00007FF71967A136+708806]
	(No symbol) [0x00007FF71955CB0F]
	(No symbol) [0x00007FF719557D1